In [1]:
from astropy.table import Table
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import h5py
import matplotlib.pyplot as plt
from gaiaxpy import generate, PhotometricSystem
import pandas as pd

In [2]:
# data = Table.read('/arc/home/aydanmckay/gaiahike/bp_rp_apogee.fits').to_pandas()
# with pd.option_context('mode.use_inf_as_na', True):
#     data.dropna(inplace=True)

In [3]:
# labels = ["bp_1", "bp_2", "bp_3", "bp_4", "bp_5", "bp_6", "bp_7",
#           "bp_8", "bp_9", "bp_10", "bp_11", "bp_12", "bp_13", "bp_14",
#           "bp_15", "bp_16", "bp_17", "bp_18", "bp_19", "bp_20", "bp_21",
#           "bp_22", "bp_23", "bp_24", "bp_25", "bp_26", "bp_27", "bp_28",
#           "bp_29", "bp_30", "bp_31", "bp_32", "bp_33", "bp_34", "bp_35",
#           "bp_36", "bp_37", "bp_38", "bp_39", "bp_40", "bp_41", "bp_42",
#           "bp_43", "bp_44", "bp_45", "bp_46", "bp_47", "bp_48", "bp_49",
#           "bp_50", "bp_51", "bp_52", "bp_53", "bp_54", "bp_55", "rp_1",
#           "rp_2", "rp_3", "rp_4", "rp_5", "rp_6", "rp_7", "rp_8",
#           "rp_9", "rp_10", "rp_11", "rp_12", "rp_13", "rp_14", "rp_15",
#           "rp_16", "rp_17", "rp_18", "rp_19", "rp_20", "rp_21", "rp_22",
#           "rp_23", "rp_24", "rp_25", "rp_26", "rp_27", "rp_28", "rp_29",
#           "rp_30", "rp_31", "rp_32", "rp_33", "rp_34", "rp_35", "rp_36",
#           "rp_37", "rp_38", "rp_39", "rp_40", "rp_41", "rp_42", "rp_43",
#           "rp_44", "rp_45", "rp_46", "rp_47", "rp_48", "rp_49", "rp_50",
#           "rp_51", "rp_52", "rp_53", "rp_54", "rp_55"]
# elabels = ["bpe_1", "bpe_2", "bpe_3", "bpe_4", "bpe_5", "bpe_6", "bpe_7",
#           "bpe_8", "bpe_9", "bpe_10", "bpe_11", "bpe_12", "bpe_13", "bpe_14",
#           "bpe_15", "bpe_16", "bpe_17", "bpe_18", "bpe_19", "bpe_20", "bpe_21",
#           "bpe_22", "bpe_23", "bpe_24", "bpe_25", "bpe_26", "bpe_27", "bpe_28",
#           "bpe_29", "bpe_30", "bpe_31", "bpe_32", "bpe_33", "bpe_34", "bpe_35",
#           "bpe_36", "bpe_37", "bpe_38", "bpe_39", "bpe_40", "bpe_41", "bpe_42",
#           "bpe_43", "bpe_44", "bpe_45", "bpe_46", "bpe_47", "bpe_48", "bpe_49",
#           "bpe_50", "bpe_51", "bpe_52", "bpe_53", "bpe_54", "bpe_55", "rpe_1",
#           "rpe_2", "rpe_3", "rpe_4", "rpe_5", "rpe_6", "rpe_7", "rpe_8",
#           "rpe_9", "rpe_10", "rpe_11", "rpe_12", "rpe_13", "rpe_14", "rpe_15",
#           "rpe_16", "rpe_17", "rpe_18", "rpe_19", "rpe_20", "rpe_21", "rpe_22",
#           "rpe_23", "rpe_24", "rpe_25", "rpe_26", "rpe_27", "rpe_28", "rpe_29",
#           "rpe_30", "rpe_31", "rpe_32", "rpe_33", "rpe_34", "rpe_35", "rpe_36",
#           "rpe_37", "rpe_38", "rpe_39", "rpe_40", "rpe_41", "rpe_42", "rpe_43",
#           "rpe_44", "rpe_45", "rpe_46", "rpe_47", "rpe_48", "rpe_49", "rpe_50",
#           "rpe_51", "rpe_52", "rpe_53", "rpe_54", "rpe_55"]

In [4]:
# for label in elabels:
#     data = data[(data[label] < 10) & (data[label] > -10)]
# data.shape
# data = data.sample(n=50000,random_state=42)

In [5]:
# scaler = StandardScaler()

In [6]:
# for elabel,label in zip(elabels,labels):
#     xp.append(scaler.fit_transform(data[[label]]).flatten())
#     xpe.append(scaler.fit_transform(data[[elabel]]).flatten())

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(3, 64),
            nn.LeakyReLU(),
            nn.Linear(64, 128),
            nn.LeakyReLU(),
            nn.Linear(128, 256),
            nn.LeakyReLU(),
            nn.Linear(256, 512),
            nn.LeakyReLU(),
            nn.Linear(512, 110)
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits
    
# defining the Dataset class
class train_set(Dataset):
    def __init__(self,file):
        fn = h5py.File(file, 'r')
        self.f = fn
        
        # get data
        dset = fn['group_1']['data']
        self.x = torch.Tensor(dset[:].T)
        
        # get label
        ydset = self.f['group_1']['label']
        self.y = torch.Tensor(ydset[:].T)
        # torch.from_numpy(y[index]) does not work since y is doubles and not floats.
        
        # get error in label # comment out for non-error label runs
        errdset = self.f['group_1']['e_label']
        self.err = torch.Tensor(errdset[:].T)
        
    def __len__(self):
        return self.f['group_1']['data'].shape[1]
  
    def __getitem__(self, index):
        xg = self.x[index]
        yg = self.y[index]
        errg = self.err[index]
        return (xg,yg,errg)

class test_set(Dataset):
    def __init__(self,file):
        fn = h5py.File(file, 'r')
        self.f = fn
        
        # get data
        dset = self.f['group_2']['data']
        self.x = torch.Tensor(dset[:].T)
        
        # get label
        ydset = self.f['group_2']['label']
        self.y = torch.Tensor(ydset[:].T)
        # torch.from_numpy(y[index]) does not work since y is doubles and not floats.
        
        # get error in label # comment out for non-error label runs
        errdset = self.f['group_2']['e_label']
        self.err = torch.Tensor(errdset[:].T)
        
    def __len__(self):
        return self.f['group_2']['data'].shape[1]
  
    def __getitem__(self, index):
        xg = self.x[index]
        yg = self.y[index]
        errg = self.err[index]
        return (xg,yg,errg)

cuda


In [8]:
# training_data = train_set("/arc/home/aydanmckay/mydataelabelssmallscalecuts.h5")
data = test_set("/arc/home/aydanmckay/elabelssmallcutscaled.h5")
loaded_data = DataLoader(
    data,
    batch_size=256,
    num_workers=4
    # shuffle=True
)

In [9]:
model = Net()
model.load_state_dict(torch.load("/arc/home/aydanmckay/torchmodel/torchmodelL2smallscalecutsbl256lr-2SGDep100load.pth"))
model.eval()

Net(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=3, out_features=64, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=64, out_features=128, bias=True)
    (3): LeakyReLU(negative_slope=0.01)
    (4): Linear(in_features=128, out_features=256, bias=True)
    (5): LeakyReLU(negative_slope=0.01)
    (6): Linear(in_features=256, out_features=512, bias=True)
    (7): LeakyReLU(negative_slope=0.01)
    (8): Linear(in_features=512, out_features=110, bias=True)
  )
)

In [10]:
model = model.to(device)

In [11]:
preds = []
# covbp = np.zeros((55,55))
# covrp = np.zeros((55,55))
covbs = []
covrs = []
with torch.no_grad():
    for X, y, z in loaded_data:
        X = X.to(device)
        y = y.to(device)
        z = z.to(device)
        pred = model(X)
        for prediction,err in zip(pred,z):
            covbp = np.zeros((55,55))
            covrp = np.zeros((55,55))
            preds.append(prediction)
            for it in range(len(err[:55])):
                covbp[it][it] += err[it].item()**2
                covrp[it][it] += err[it+55].item()**2
            covbs.append(covbp)
            covrs.append(covrp)
        # preds.append(pred)
        # for m in z:
        #     for i in range(len(m[:55])):
        #         covbp[i][i] += m[i].item()**2
        #         covrp[i][i] += m[55+i].item()**2
        #     break
        # break

In [12]:
np.array(covrs).shape

(5000, 55, 55)

In [13]:
# np.array(preds[4000].numpy()).shape

In [14]:
# PhotometricSystem.get_available_systems()

In [15]:
phot_system_list = [
    PhotometricSystem.Gaia_2,
    PhotometricSystem.Gaia_DR3_Vega,
    PhotometricSystem.PanSTARRS1,
    PhotometricSystem.PanSTARRS1_Std,
    PhotometricSystem.Pristine,
    PhotometricSystem.SDSS,
    PhotometricSystem.SDSS_Std
]

In [16]:
# preds[0][0]

In [17]:
# len(preds[0][0].to('cpu').numpy()[55:])

In [18]:
df = pd.DataFrame(
    {'source_id':range(len(preds)),
     'bp_coefficients':[pred.to('cpu').numpy()[:55] for pred in preds],
     'bp_standard_deviation':[np.std(pred.to('cpu').numpy()[:55]) for pred in preds],
     'bp_coefficient_covariances':covbs,
     'rp_coefficients':[pred.to('cpu').numpy()[55:] for pred in preds],
     'rp_coefficient_covariances':covrs,
     'rp_standard_deviation':[np.std(pred.to('cpu').numpy()[55:]) for pred in preds]
    }
)

In [19]:
df

,source_id,bp_coefficients,bp_standard_deviation,bp_coefficient_covariances,rp_coefficients,rp_coefficient_covariances,rp_standard_deviation
0,0,"[-0.29479945, 0.46684635, -0.111822665, -0.113...",0.277531,"[[1.9351570247322911, 0.0, 0.0, 0.0, 0.0, 0.0,...","[-0.17784107, 0.31998456, 0.053711902, 0.39122...","[[0.7542551756064455, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.179478
1,1,"[1.1239067, -2.320655, 1.2068859, 0.14509504, ...",0.804312,"[[2.4918486459962566, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.45289925, -1.2885636, -1.0829724, -0.959804...","[[0.2337432340115706, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.366829
2,2,"[-0.6471436, 0.7551672, 0.370026, -0.82528156,...",0.390244,"[[1.4512986267836823, 0.0, 0.0, 0.0, 0.0, 0.0,...","[-0.28900838, 0.9096695, 0.5968928, 0.54394174...","[[2.078506742956293, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.356175
3,3,"[-0.5411881, 0.57020205, 0.20330904, -1.059089...",0.474140,"[[0.5597425006308185, 0.0, 0.0, 0.0, 0.0, 0.0,...","[-0.4065677, 0.34198874, 0.7502882, -0.2655033...","[[0.8143704343077651, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.348839
4,4,"[-0.12166545, 0.72125727, -0.40895867, -0.4179...",0.311334,"[[4.074725694249594, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.17515668, 0.4432059, 0.1738915, 0.9320497, ...","[[4.112673264691921, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.279279
...,...,...,...,...,...,...,...
4995,4995,"[0.120846, -0.35384366, 0.04182273, 0.68377376...",0.391667,"[[1.127633677137979, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.060011692, -0.36167026, -0.2793406, -0.295...","[[0.3623379778378535, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.169727
4996,4996,"[-0.32566896, 0.7816641, -0.18934804, -0.66036...",0.346978,"[[0.38313711407599627, 0.0, 0.0, 0.0, 0.0, 0.0...","[-0.00068360753, 0.62012535, 0.20865157, 0.993...","[[0.04858355879520104, 0.0, 0.0, 0.0, 0.0, 0.0...",0.292068
4997,4997,"[0.15535156, -0.18826133, -0.25277847, 0.67818...",0.299169,"[[4.421313179264018, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.08628062, -0.30231813, -0.19291364, -0.0868...","[[3.4547481016311963, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.125075
4998,4998,"[-0.10169849, 0.7946698, -0.585961, -0.5864640...",0.547077,"[[1.6086364392913879, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.1380955, 0.33508286, 0.033051837, 1.1820123...","[[0.8577394037549766, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.560528


In [20]:
f = '/arc/home/aydanmckay/XpContinuousMeanSpectrum_407725-409897.csv'

In [21]:
# synthetic_photometry = generate(f, photometric_system=phot_system_list)
synthetic_photometry = generate(df, photometric_system=PhotometricSystem.Pristine)
synthetic_photometry

,source_id,Pristine_mag_CaHK,Pristine_flux_CaHK,Pristine_flux_error_CaHK
0,0,21.380412,2.042280e-19,5.463393e-19
1,1,NaN,-3.273722e-18,2.700813e-18
2,2,21.294285,2.210884e-19,6.340796e-19
3,3,22.762500,5.718470e-20,5.527748e-19
4,4,21.050847,2.766568e-19,6.973574e-19
...,...,...,...,...
4995,4995,NaN,-5.238686e-20,7.452216e-19
4996,4996,21.016300,2.856012e-19,4.060348e-19
4997,4997,20.560757,4.344876e-19,7.803014e-19
4998,4998,21.029694,2.820996e-19,8.739111e-19


In [22]:
data = Table.read('/arc/home/aydanmckay/gaiahike/bp_rp_apogee.fits').to_pandas()

In [23]:
with pd.option_context('mode.use_inf_as_na', True):
    data.dropna(inplace=True)
data.shape

(481813, 230)

In [24]:
labels = ["bp_1", "bp_2", "bp_3", "bp_4", "bp_5", "bp_6", "bp_7",
          "bp_8", "bp_9", "bp_10", "bp_11", "bp_12", "bp_13", "bp_14",
          "bp_15", "bp_16", "bp_17", "bp_18", "bp_19", "bp_20", "bp_21",
          "bp_22", "bp_23", "bp_24", "bp_25", "bp_26", "bp_27", "bp_28",
          "bp_29", "bp_30", "bp_31", "bp_32", "bp_33", "bp_34", "bp_35",
          "bp_36", "bp_37", "bp_38", "bp_39", "bp_40", "bp_41", "bp_42",
          "bp_43", "bp_44", "bp_45", "bp_46", "bp_47", "bp_48", "bp_49",
          "bp_50", "bp_51", "bp_52", "bp_53", "bp_54", "bp_55", "rp_1",
          "rp_2", "rp_3", "rp_4", "rp_5", "rp_6", "rp_7", "rp_8",
          "rp_9", "rp_10", "rp_11", "rp_12", "rp_13", "rp_14", "rp_15",
          "rp_16", "rp_17", "rp_18", "rp_19", "rp_20", "rp_21", "rp_22",
          "rp_23", "rp_24", "rp_25", "rp_26", "rp_27", "rp_28", "rp_29",
          "rp_30", "rp_31", "rp_32", "rp_33", "rp_34", "rp_35", "rp_36",
          "rp_37", "rp_38", "rp_39", "rp_40", "rp_41", "rp_42", "rp_43",
          "rp_44", "rp_45", "rp_46", "rp_47", "rp_48", "rp_49", "rp_50",
          "rp_51", "rp_52", "rp_53", "rp_54", "rp_55"]
elabels = ["bpe_1", "bpe_2", "bpe_3", "bpe_4", "bpe_5", "bpe_6", "bpe_7",
          "bpe_8", "bpe_9", "bpe_10", "bpe_11", "bpe_12", "bpe_13", "bpe_14",
          "bpe_15", "bpe_16", "bpe_17", "bpe_18", "bpe_19", "bpe_20", "bpe_21",
          "bpe_22", "bpe_23", "bpe_24", "bpe_25", "bpe_26", "bpe_27", "bpe_28",
          "bpe_29", "bpe_30", "bpe_31", "bpe_32", "bpe_33", "bpe_34", "bpe_35",
          "bpe_36", "bpe_37", "bpe_38", "bpe_39", "bpe_40", "bpe_41", "bpe_42",
          "bpe_43", "bpe_44", "bpe_45", "bpe_46", "bpe_47", "bpe_48", "bpe_49",
          "bpe_50", "bpe_51", "bpe_52", "bpe_53", "bpe_54", "bpe_55", "rpe_1",
          "rpe_2", "rpe_3", "rpe_4", "rpe_5", "rpe_6", "rpe_7", "rpe_8",
          "rpe_9", "rpe_10", "rpe_11", "rpe_12", "rpe_13", "rpe_14", "rpe_15",
          "rpe_16", "rpe_17", "rpe_18", "rpe_19", "rpe_20", "rpe_21", "rpe_22",
          "rpe_23", "rpe_24", "rpe_25", "rpe_26", "rpe_27", "rpe_28", "rpe_29",
          "rpe_30", "rpe_31", "rpe_32", "rpe_33", "rpe_34", "rpe_35", "rpe_36",
          "rpe_37", "rpe_38", "rpe_39", "rpe_40", "rpe_41", "rpe_42", "rpe_43",
          "rpe_44", "rpe_45", "rpe_46", "rpe_47", "rpe_48", "rpe_49", "rpe_50",
          "rpe_51", "rpe_52", "rpe_53", "rpe_54", "rpe_55"]

In [25]:
for label in elabels:
    data = data[(data[label] < 10) & (data[label] > -10)]
data.shape

(319316, 230)

In [26]:
data = data.sample(n=50000,random_state=11)

In [30]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [32]:
xp = []
xpe = []
for elabel,label in zip(elabels,labels):
    scaled = scaler.fit(data[[label]])
    xp.append(scaled)
    scaled = scaler.fit(data[[elabel]])
    xpe.append(scaled)
    # xp.append(data[label])
    # xpe.append(data[elabel])
xp = np.array(xp)
xpe = np.array(xpe)

In [33]:
xp

array([StandardScaler(), StandardScaler(), StandardScaler(),
       StandardScaler(), StandardScaler(), StandardScaler(),
       StandardScaler(), StandardScaler(), StandardScaler(),
       StandardScaler(), StandardScaler(), StandardScaler(),
       StandardScaler(), StandardScaler(), StandardScaler(),
       StandardScaler(), StandardScaler(), StandardScaler(),
       StandardScaler(), StandardScaler(), StandardScaler(),
       StandardScaler(), StandardScaler(), StandardScaler(),
       StandardScaler(), StandardScaler(), StandardScaler(),
       StandardScaler(), StandardScaler(), StandardScaler(),
       StandardScaler(), StandardScaler(), StandardScaler(),
       StandardScaler(), StandardScaler(), StandardScaler(),
       StandardScaler(), StandardScaler(), StandardScaler(),
       StandardScaler(), StandardScaler(), StandardScaler(),
       StandardScaler(), StandardScaler(), StandardScaler(),
       StandardScaler(), StandardScaler(), StandardScaler(),
       StandardScaler(),